In [1]:
import os
import json
import networkx as nx
import pandas as pd
from google.cloud import bigquery
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pathlib import Path

part_path = Path("part-2")
part_path.mkdir(exist_ok=True)

raw_path = Path(f"{part_path}/raw")
raw_path.mkdir(exist_ok=True)

processed_path = Path(f"{part_path}/processed")
processed_path.mkdir(exist_ok=True)

submission_path = Path(f"{part_path}/submission")
submission_path.mkdir(exist_ok=True)

In [3]:
print(raw_path)

part-2/raw


In [4]:
# replace with your path to credentials
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../oso_gcp_credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('GCP_CREDENTIAL_PATH')
# replace with your project name
# client = bigquery.Client(project='opensource-observer')
client = bigquery.Client(project=os.getenv('GCP_PROJECT_ID'))

In [5]:
def stringify_array(arr):
    return "'" + "','".join(arr) + "'"

In [6]:
def remove_str(projects):
    projects = [p.replace('https://github.com/', '') for p in projects]
    return projects

In [7]:
def remove_str_df(df):
    df['project_a'] = df['project_a'].str.replace('https://github.com/', '')
    df['project_b'] = df['project_b'].str.replace('https://github.com/', '')
    return df


In [8]:
def get_dataset():
    if str(part_path) == 'part-1':
        # repository_url = (
        #     "https://raw.githubusercontent.com/deepfunding/mini-contest/refs/heads/main/"
        # )
        
        # df_train = pd.read_csv(f"{repository_url}/dataset.csv")
        # df_test = pd.read_csv(f"{repository_url}/test.csv")
        # df_train.to_csv(f"{part_path}/train.csv", index=False)
        # df_test.to_csv(f"{part_path}/test.csv", index=False)
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
        
    elif str(part_path) == 'part-2':
        df_train = pd.read_csv(f"{part_path}/train.csv")
        df_test = pd.read_csv(f"{part_path}/test.csv")
     
    return df_train, df_test

In [9]:
df_train, df_test = get_dataset()

In [10]:
len(df_train), len(df_test)

(20958, 4261)

In [11]:
projects = pd.concat([
    df_train['project_a'],
    df_train['project_b'],
    df_test['project_a'],
    df_test['project_b']
]).unique().tolist()

In [12]:
projects[:5]

['https://github.com/mochajs/mocha',
 'https://github.com/chzyer/readline',
 'https://github.com/gulpjs/gulp',
 'https://github.com/webpack/webpack',
 'https://github.com/redux-saga/redux-saga']

In [13]:
len(projects)

117

In [14]:
def get_repo_data(projects):
    headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
    "X-GitHub-Api-Version": "2022-11-28",
    }

    data = []
    for project in projects:
        api_url = f"https://api.github.com/repos/{project}"
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data.append(response.json())
    df_projects = pd.DataFrame(data)
    df_projects.to_csv(f"{raw_path}/github-projects.csv", index=False)

    return df_projects

In [15]:
trimed_projects = remove_str(projects)
df_projects = get_repo_data(trimed_projects)


In [ ]:
trimed_projects

In [ ]:
headers = {
"Accept": "application/vnd.github+json",
"Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
"X-GitHub-Api-Version": "2022-11-28",
}

api_url = f"https://api.github.com/repos/{trimed_projects[0]}/activity?time_period='year'"
response = requests.get(api_url, headers=headers)
response.raise_for_status()
res = response.json()
res


In [48]:
len(res)

30

In [ ]:
def get_repo_data_by_time(projects):
    headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {os.getenv('GITHUB_TOKEN')}",
    "X-GitHub-Api-Version": "2022-11-28",
    }

    data = []
    for project in projects:
        api_url = f"https://api.github.com/repos/{project}"
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        data.append(response.json())
    df_projects = pd.DataFrame(data)
    df_projects.to_csv(f"{raw_path}/github-projects.csv", index=False)

    return df_projects

In [18]:
q = f"""
    WITH repos as(
    SELECT
        artifact_id,
        artifact_url AS repo_url,
    FROM
        `oso_production.repositories_v0`
    ),
    events as(
    SELECT
        artifact_id,
        event_type,
        CAST(sum(amount) AS FLOAT64) AS total_amount
    FROM
        `oso_production.events_monthly_to_artifact`
    WHERE bucket_month >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 180 DAY)
    GROUP BY 1,2
    )
    SELECT
        repos.repo_url,
        events.event_type,
        events.total_amount
    FROM repos
    JOIN events
        ON repos.artifact_id = events.artifact_id
    WHERE repo_url IN ({stringify_array(projects)})

    """

results = client.query(q)
df = results.to_dataframe()

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [49]:
q = f"""
    SELECT
        *
    FROM  `oso_production.int_events`
    --WHERE repo_artifact_id IN ('-FF6BVMZVV5G12UjcHMP9LpHMGZJNfzr0ykne9dsrcs=')
    LIMIT 10
    """

results = client.query(q)
df = results.to_dataframe()
df.tail()

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,time,to_artifact_id,from_artifact_id,event_type,event_source_id,event_source,to_artifact_name,to_artifact_namespace,to_artifact_type,to_artifact_source_id,from_artifact_name,from_artifact_namespace,from_artifact_type,from_artifact_source_id,amount
5,2018-01-08 18:03:04.439000+00:00,sq8PASwibu5FkLhTMbkQAtKrgiPSG0UWaz-5V2xJol8=,q3bfsbBdxyhRVbl-AjwDbCZbwUv5Op15Q7kdiSmiIaQ=,DEBIT,9026a88a-5c56-456f-b579-425af451e588,OPEN_COLLECTIVE,qubes os,qubes-os,COLLECTIVE,obvrgbk3-57l4x96e-4jpomewa-0jdyzn83,anonymous,anonymous103,INDIVIDUAL,rmvrwng4-kj03dpby-k0qz57oy-l9e8xbaj,64.27
6,2018-01-08 05:41:03.131000+00:00,bZaURi262iSe8-fyvYNCfS87l21__EysT51E1PIYKvc=,OSkBKkVtQ6s-34BlKYwNQOoVW5lgeCBbyZoS2QNtMks=,CREDIT,4da9fff4-97d5-4295-9fe4-ccdcee328961,OPEN_COLLECTIVE,spyder ide,spyder,COLLECTIVE,zzaxon79-3jy8gplg-e36brkde-mwl5v04o,anonymous,anonymous97,INDIVIDUAL,ggnxdwzj-3le5mpwo-ej6y8rvb-ko04a97b,50.00
7,2018-01-08 05:41:03.119000+00:00,bZaURi262iSe8-fyvYNCfS87l21__EysT51E1PIYKvc=,OSkBKkVtQ6s-34BlKYwNQOoVW5lgeCBbyZoS2QNtMks=,DEBIT,889c05bd-ce9b-4edc-8fa6-a84e459638c9,OPEN_COLLECTIVE,spyder ide,spyder,COLLECTIVE,zzaxon79-3jy8gplg-e36brkde-mwl5v04o,anonymous,anonymous97,INDIVIDUAL,ggnxdwzj-3le5mpwo-ej6y8rvb-ko04a97b,43.25
8,2018-01-08 18:40:47.247000+00:00,-8s1kBpLbP4YtApqkpwbXENaq92G__UWKp6Bvg6HzFc=,qxWLWbrXAHHDbDUgWUOt4uzzgo1iXVbmz-r510caIXs=,CREDIT,8e7fe603-0a89-4db8-9fbc-cc954a0a9531,OPEN_COLLECTIVE,streetmix,streetmix,COLLECTIVE,oeejoxl3-az45w9pa-9eqy870m-gkbrvdn8,cam solomon,cam-solomon,INDIVIDUAL,ggnxdwzj-3le5mpwo-4j6y8rvb-ko04a97b,10.00
9,2018-01-08 18:40:47.232000+00:00,-8s1kBpLbP4YtApqkpwbXENaq92G__UWKp6Bvg6HzFc=,qxWLWbrXAHHDbDUgWUOt4uzzgo1iXVbmz-r510caIXs=,DEBIT,28fc6f76-b866-44ea-9919-671641e66911,OPEN_COLLECTIVE,streetmix,streetmix,COLLECTIVE,oeejoxl3-az45w9pa-9eqy870m-gkbrvdn8,cam solomon,cam-solomon,INDIVIDUAL,ggnxdwzj-3le5mpwo-4j6y8rvb-ko04a97b,8.41


In [52]:
projects[:5]


['https://github.com/mochajs/mocha',
 'https://github.com/chzyer/readline',
 'https://github.com/gulpjs/gulp',
 'https://github.com/webpack/webpack',
 'https://github.com/redux-saga/redux-saga']

In [55]:
q = f"""
    SELECT
        *
    FROM  `oso_production.timeseries_events_aux_issues_by_artifact_v0`
    --WHERE artifact_url IN ({stringify_array(projects)})
    LIMIT 1000
    """

results = client.query(q)
df = results.to_dataframe()
df.tail()

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,time,to_artifact_id,from_artifact_id,event_type,event_source_id,event_source,issue_id,issue_number,created_at,merged_at,closed_at,comments
995,2023-08-30 22:09:00+00:00,-ekQkLmq8I5Fc3rnAwDeZvi-XGumtxQlQF6_5z7Pajs=,iDEK2pJDr0Bzj_GsKgpRJ24eWRBJVWJlIaQjR1btdXw=,ISSUE_CLOSED,31492728507,GITHUB,TzJCSsBOLKRVWVRtBv2pgy0WhwwQZiIPUVc8r-RTdcM=,13,2023-08-06 06:55:13+00:00,NaT,2023-08-30 22:08:59+00:00,4
996,2023-11-29 17:23:17+00:00,-ekQkLmq8I5Fc3rnAwDeZvi-XGumtxQlQF6_5z7Pajs=,gl7urTu544OCLKVxV5lXGhGgbJkoPl7JGlYVfh0qYDw=,ISSUE_COMMENT,33727171406,GITHUB,DKyUg86I6kbI12dcQ0qDXkOG-Zb7MNA8N4qiCyxUCJE=,41,2023-11-29 16:08:06+00:00,NaT,NaT,5
997,2023-08-23 16:39:23+00:00,-ekQkLmq8I5Fc3rnAwDeZvi-XGumtxQlQF6_5z7Pajs=,rY1CWSWpe6CyDJSZIMSvh4nWoydxUzitzEKmQJ4G1-Q=,PULL_REQUEST_MERGED,1486734671,GITHUB,t5pNOYbYwtB8Y4YWffhlv3Mtdt0ujbOgpVuDW0Xs-og=,20,2023-08-23 16:39:23+00:00,2023-08-25 22:30:32+00:00,2023-08-25 22:30:32+00:00,0
998,2024-09-09 23:19:54+00:00,-fFLbGpqSm-CdAuhjSP-XN32foUjGUHmSCriv91ARE0=,0ykkTNnozNS0rvasx6ujldAf9C-hlZBN1ryn1mzq1b8=,PULL_REQUEST_MERGED,2062122603,GITHUB,KrzJ5Gqont7VJHWyHGiMbR1anNN39xHlrWdnWr4zA1k=,25,2024-09-09 23:19:54+00:00,2024-09-09 23:25:07+00:00,2024-09-09 23:25:07+00:00,0
999,2024-08-21 02:26:39+00:00,-fFLbGpqSm-CdAuhjSP-XN32foUjGUHmSCriv91ARE0=,kLNI82Fp3Dv6hyNVH9xW7pUL5HqI-N5sNHixH-6OenY=,PULL_REQUEST_OPENED,41195081128,GITHUB,01q_lMlP5b5jOiwPQcqP4wuj4yF46iGlA4kgXoEw_PI=,22,2024-08-21 02:26:38+00:00,NaT,NaT,0


In [36]:
def get_repo_activity(urls):
    
    q = f"""
    WITH repos as(
    SELECT
        artifact_id,
        artifact_url AS repo_url,
    FROM
        `oso_production.repositories_v0`
    ),
    events as(
    SELECT
        artifact_id,
        event_type,
        CAST(sum(amount) AS FLOAT64) AS total_amount
    FROM
        `oso_production.events_monthly_to_artifact`
    WHERE bucket_month >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 180 DAY)
    GROUP BY 1,2
    )
    SELECT
        repos.repo_url,
        events.event_type,
        events.total_amount
    FROM repos
    JOIN events
        ON repos.artifact_id = events.artifact_id
    WHERE repo_url IN ({stringify_array(urls)})

    """

    results = client.query(q)
    df = results.to_dataframe()
    df_pivot = df.pivot_table(index=['repo_url'], columns='event_type', values='total_amount', fill_value=0)
    df_pivot.reset_index(inplace=True)
    df_pivot.to_csv(f"{raw_path}/github-activity.csv", index=False)
    return df_pivot

In [37]:
df_act = get_repo_activity(projects)

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [38]:
with open("unweighted_graph.json", 'r') as f:
    graph_data = json.load(f)

G_original = nx.node_link_graph(graph_data)
print("Nodes:", len(G_original.nodes))
repo_urls = [x for x in G_original.nodes]
print("Edges:", len(G_original.edges))

Nodes: 5235
Edges: 17367


/home/jake/deepfunding/.venv/lib/python3.13/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


In [39]:
def get_repo_metrics_and_metadata(urls):
    query = f"""
    WITH repos AS (
    SELECT *
    FROM `oso_production.repositories_v0`
    ),
    package_owners AS (
    SELECT
        package_owner_artifact_id,
        package_artifact_source,
        package_artifact_name,
        CONCAT(package_artifact_source, '/', package_artifact_name) AS package_tag
    FROM `oso_production.package_owners_v0`
    WHERE package_owner_artifact_id IN (SELECT artifact_id FROM repos)
    ),
    oso_dependents AS (
    SELECT
        package_owners.package_owner_artifact_id,
        COUNT(DISTINCT package_owners.package_tag) AS num_packages,
        COUNT(DISTINCT sboms.from_artifact_namespace) AS num_dependents_in_oso,
        ARRAY_AGG(DISTINCT package_owners.package_tag) AS list_of_packages,
        ARRAY_AGG(DISTINCT sboms.from_artifact_namespace) AS list_of_dependents_in_oso
    FROM `oso_production.sboms_v0` AS sboms
    JOIN package_owners
        ON sboms.to_package_artifact_name = package_owners.package_artifact_name
        AND sboms.to_package_artifact_source = package_owners.package_artifact_source
    GROUP BY 1
    ),
    grants AS (
    SELECT
        funding.to_project_id AS project_id,
        ARRAY_AGG(DISTINCT projects.display_name) AS list_of_funders,
        SUM(funding.amount) AS total_funding_usd,
        SUM(CASE WHEN funding.time > '2023-01-01' THEN funding.amount ELSE 0 END) AS total_funding_usd_since_2023
    FROM `oso_production.oss_funding_v0` AS funding
    JOIN `oso_production.projects_v1` AS projects
        ON funding.from_project_id = projects.project_id
    WHERE funding.from_project_name IN ('gitcoin', 'octant-golemfoundation', 'opencollective', 'optimism')
    GROUP BY 1
    ),
    combined AS (
    SELECT
        repos.artifact_url AS repo_url,
        repos.artifact_namespace AS maintainer,
        repos.language,
        repos.is_fork,
        DATE(repos.created_at) as created_at,
        DATE(repos.updated_at) as updated_at,
        repos.star_count,
        repos.fork_count,
        COALESCE(oso_dependents.num_packages, 0) AS num_packages,
        COALESCE(oso_dependents.num_dependents_in_oso, 0) AS num_dependents_in_oso,
        oso_dependents.list_of_dependents_in_oso,
        oso_dependents.list_of_packages,
        grants.list_of_funders,
        COALESCE(grants.total_funding_usd, 0) AS total_funding_usd,
        COALESCE(grants.total_funding_usd_since_2023, 0) AS total_funding_usd_since_2023
    FROM repos
    LEFT JOIN oso_dependents
        ON repos.artifact_id = oso_dependents.package_owner_artifact_id
    LEFT JOIN grants
        ON repos.project_id = grants.project_id
    )
    SELECT
    *,
    PERCENT_RANK() OVER (ORDER BY num_dependents_in_oso) AS oso_dependency_rank,
    COUNT(*) OVER (PARTITION BY language) AS num_repos_in_same_language,
    PERCENT_RANK() OVER (PARTITION BY language ORDER BY num_dependents_in_oso) AS oso_dependency_rank_for_language
    FROM combined
    WHERE repo_url IN ({stringify_array(repo_urls)})

    """

    # execute the query and save it
    results = client.query(query)
    df = results.to_dataframe()
    df.to_csv(f"{raw_path}/repo_metrics_and_metadata.csv")
    print("Query saved to local csv file.")
 
    return df


In [40]:
df_repo = get_repo_metrics_and_metadata(repo_urls)

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Query saved to local csv file.


In [41]:
def get_dependent_repos_in_OSO():
  query = """
  select
    p.project_id,
    pkgs.package_artifact_source,
    pkgs.package_artifact_name,
    count(distinct sboms.from_project_id) as num_dependents
  from `oso_production.package_owners_v0` pkgs
  join `oso_production.sboms_v0` sboms
    on pkgs.package_artifact_name = sboms.to_package_artifact_name
    and pkgs.package_artifact_source = sboms.to_package_artifact_source
  join `oso_production.projects_v1` p
    on pkgs.package_owner_project_id = p.project_id
  where pkgs.package_owner_project_id is not null
  group by 1,2,3
  order by 4 desc
  """

  results = client.query(query)
  df = results.to_dataframe()
  df.to_csv(f"{raw_path}/dependent-metrics.csv")
  print("Query saved to local csv file.")
  
  return df

In [42]:
df_dependent = get_dependent_repos_in_OSO()

/home/jake/deepfunding/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Query saved to local csv file.
